<a href="https://colab.research.google.com/github/moey920/NLP/blob/master/GloVe_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 전처리

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
train_file_link = '/content/drive/My Drive/datt/CNN_텍스트분류_정답_수정.txt'

In [0]:
# 불러온 데이터를 보면 id, document, label로 구분이 되어있습니다.
train_data = pd.read_csv(train_file_link, header = 0, delimiter = '\t', quoting = 3)
train_data.head(10)

In [0]:
!pip install konlpy

In [0]:
import re
import json
from konlpy.tag import Okt

from tqdm import tqdm

In [0]:
def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    # 함수의 인자는 다음과 같다.
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    # remove_stopword : 불용어를 제거할지 선택 기본값은 False
    # stop_word : 불용어 사전은 사용자가 직접 입력해야함 기본값은 비어있는 리스트
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거. + 영어 소문자, 대문자, 숫자도 제외
    # 일단 OCR 결과의 원형을 학습시키기 위해 정규표현식을 사용하지 않고 학습시켜보겠습니다.
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\\s]", " ",  review)
    #review_text = re.sub(" ", "",  review)
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    word_review = okt.morphs(review_text, stem=True)
    
    if remove_stopwords:
        
        # 불용어 제거(선택적)
        word_review = [token for token in word_review if not token in stop_words]
        
   
    return word_review

In [0]:
stop_words = ['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', 
              '주', '등', '한', '(', ')', '/', '*', '=', 'E', '|', '-', '.', ',', 'II', 'لالالالا', 
              '|||||||||', 'iii', '|||', '. ', '.', '"', ' )', '[', ']', '"']
okt = Okt()
clean_train_review = []

for review in tqdm(train_data['document']):
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_train_review.append(preprocessing(review, okt, remove_stopwords = True, stop_words=stop_words))
    else:
        clean_train_review.append([])  #string이 아니면 비어있는 값 추가

In [0]:
sentences = clean_train_review
y_train = train_data['label']

문장과 레이블 데이터를 만들었습니다. 긍정인 문장은 레이블 1, 부정인 문장은 레이블이 0입니다.

In [0]:
t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1

print(vocab_size)

케라스의 Tokenizer()를 사용하여 토큰화를 시켰습니다.

In [0]:
X_encoded = t.texts_to_sequences(sentences)
print(X_encoded)

각 문장에 대해서 정수 인코딩을 수행합니다.

In [0]:
max_len=max(len(l) for l in X_encoded)
print(max_len)

문장 중에서 가장 길이가 긴 문장의 길이는 45입니다.

In [0]:
X_train=pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train=np.array(y_train)
print(X_train)
print(y_train)

In [0]:
input_data = X_train
label_data = y_train
word_index = t.word_index

모든 문장을 패딩하여 길이를 45로 만들어주었습니다. 훈련 데이터에 대한 전처리가 끝났습니다. 모델을 설계합니다.

### 훈련데이터 훈련,검증,테스트셋 나누기

In [0]:
TEST_SPLIT = 0.1
RNG_SEED = 54648258
VOCAB_SIZE = len(word_index)+1
EMB_SIZE = 20
BATCH_SIZE = 12
NUM_EPOCHS = 10

인풋 데이터와 인풋 레이블을 앞서 설정한 변수값에 맞추어 훈련, 검증, 테스트 데이터로 나눠주도록 하겠습니다.

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

input_train, input_test, label_train, label_test = train_test_split(input_data, label_data, test_size=TEST_SPLIT, random_state=RNG_SEED)
#10%의 데이터를 테스트 데이터로 활용하도록 하겠습니다.

훈련(90%), 테스트(10%)를 나눴으니 훈련 데이터에서 일정량을 떼어 검증 데이터로 사용하겠습니다.

In [0]:
input_val = input_train[30000:]
label_val = label_train[30000:]
input_train = input_train[:30000]
label_train = label_train[:30000]

In [0]:
print(len(input_train),len(label_train),': 훈련용 데이터/라벨','\n',len(input_val),len(label_val),': 검증용 데이터/라벨','\n',len(input_test),len(label_test),': 테스트용 데이터/라벨')

### 패딩작업, 원핫인코딩

In [0]:
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Flatten, Dropout
from keras.layers import Conv1D, GlobalMaxPooling1D

In [0]:
# 데이터셋 전처리 : 문장 길이 맞추기
input_train = sequence.pad_sequences(input_train, maxlen=max_len)
input_val = sequence.pad_sequences(input_val, maxlen=max_len)
input_test = sequence.pad_sequences(input_test, maxlen=max_len)

#ndarray with shape(16000,120) : input_train
#ndarray with shape(2000,120) : input_val
#ndarray with shape(2000,120) : input_test

In [0]:
# one-hot 인코딩
label_train = np_utils.to_categorical(label_train)
label_val = np_utils.to_categorical(label_val)
label_test = np_utils.to_categorical(label_test)

#ndarray with shape(16000, ) : label_train

In [0]:
print(y_train)

# 사전 훈련된 워드 임베딩(Pre-trained Word Embedding)
이번엔 케라스의 임베딩 층(embedding layer)과 사전 훈련된 워드 임베딩(pre-trained word embedding)을 가져와서 사용하는 것을 비교해봅니다. 자연어 처리를 하려고 할 때 갖고 있는 훈련 데이터의 단어들을 임베딩 층(embedding layer)을 구현하여 임베딩 벡터로 학습하는 경우가 있습니다. 케라스에서는 이를 Embedding()이라는 도구를 사용하여 구현합니다.

그런데 위키피디아 등과 같은 방대한 코퍼스를 가지고 Word2vec, FastText, GloVe 등을 통해서 이미 미리 훈련된 임베딩 벡터를 불러오는 방법을 사용하는 경우도 있습니다. 이는 현재 갖고 있는 훈련 데이터를 임베딩 층으로 처음부터 학습을 하는 방법과는 대조됩니다.

##  GloVe 사용하기

이제 임베딩 층을 설계하기 위한 과정부터 달라집니다. 우선 다운로드 받은 파일인 glove.6B.zip의 압축을 풀면 그 안에 4개의 파일이 있는데 여기서 사용할 파일은 glove.6B.100d.txt 파일입니다. 해당 파일은 하나의 줄당 101개의 값을 가지는 리스트를 갖고 있습니다. 두 개의 줄만 읽어보도록 하겠습니다.

[꿀팁]Colab에서 구글 드라이브 마운트를 통해 파일 가져오기
Colab에서 구글 드라이브 API를 통해 굳이 연결하지 않더라도,
Colab 왼쪽 상단을 보면 폴더 모양의 아이콘이 있습니다. 이를 클릭하고
드라이브 마운트를 통해 계정에 연결해놓으면 개인 구글 드라이브에 있는 폴더들이 나타납니다.
이 중 content/drive/Mydrive/ 로 들어가면 자신의 구글 드라이브에 있는 폴더와 파일들이 나타납니다.
사용하고자 하는 파일을 우클릭하고 경로를 복사하면
f = open('복사한 경로 붙여넣기')를 통해
간편하게 사용할 수 있습니다.

In [0]:
n=0
f = open('/content/drive/My Drive/datt/glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split() # 각 줄을 읽어와서 word_vector에 저장.
    print(word_vector) # 각 줄을 출력
    word = word_vector[0] # word_vector에서 첫번째 값만 저장
    print(word) # word_vector의 첫번째 값만 출력
    n=n+1
    if n==2:
        break
f.close()

In [0]:
print(type(word_vector))
print(len(word_vector))

101개의 값 중에서 첫번째 값은 임베딩 벡터가 의미하는 단어를 의미하며, 
두번째 값부터 마지막 값은 해당 단어의 임베딩 벡터의 100개의 차원에서의 각 값을 의미합니다. 

즉, glove.6B.100d.txt는 수많은 단어에 대해서 100개의 차원을 가지는 임베딩 벡터로 제공하고 있습니다. 

위의 출력 결과는 단어 'the'에 대해서 100개의 차원을 가지는 임베딩 벡터와 단어 ','에 대해서 100개의 차원을 가지는 임베딩 벡터를 보여줍니다. 

그러면 이제 glove.6B.100d.txt에 있는 모든 임베딩 벡터들을 불러와보겠습니다. 

형식은 키(key)와 값(value)의 쌍(pair)를 가지는 파이썬의 사전형 구조를 사용합니다.

In [0]:
import numpy as np
embedding_dict = dict()
f = open('/content/drive/My Drive/datt/glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()
print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

임의의 단어 'respectable'에 대해서 임베딩 벡터를 출력해봅니다.

In [0]:
print(embedding_dict['respectable'])
print(len(embedding_dict['respectable']))

벡터값이 출력되며 길이는 100인 것을 확인할 수 있습니다.

In [0]:
embedding_matrix = np.zeros((vocab_size, 100))
# 단어 집합 크기의 행과 100개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

In [0]:
print(t.word_index.items())

In [0]:
for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어를 1개씩 꺼내온다.
    temp = embedding_dict.get(word) # 단어(key) 해당되는 임베딩 벡터의 100개의 값(value)를 임시 변수에 저장
    if temp is not None:
        embedding_matrix[i] = temp # 임수 변수의 값을 단어와 맵핑되는 인덱스의 행에 삽입

이제 훈련 데이터의 단어 집합의 모든 단어에 대해서 사전 훈련된 GloVe의 임베딩 벡터들을 맵핑하였습니다. 이제 이를 이용하여 임베딩 층(embedding layer)를 만들어보겠습니다.

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=True)
#e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)

현재 실습에서 사전 훈련된 워드 임베딩을 100차원의 값인 것으로 사용하고 있기 때문에 임베딩 층의 output_dim의 인자값으로 100을 주어야 합니다. 그리고 사전 훈련된 워드 임베딩을 그대로 사용할 것이므로, 별도로 더 이상 훈련을 하지 않는다는 옵션을 줍니다. 이는 trainable=False로 선택할 수 있습니다.

### 모델 생성

In [0]:
# 특정 클래스에 대한 정밀도
def single_class_precision(interesting_class_id):
    def prec(y_true, y_pred):
        class_id_true = K.argmax(y_true, axis=-1) # y_true: 실제 값, 티아노 및 텐스플로우의 텐서(tensor)
        class_id_pred = K.argmax(y_pred, axis=-1) # y_pred: 예측 값, 티아노 및 텐스플로우의 텐서(tensor)
        precision_mask = K.cast(K.equal(class_id_pred, interesting_class_id), 'int32')
        class_prec_tensor = K.cast(K.equal(class_id_true, class_id_pred), 'int32') * precision_mask
        class_prec = K.cast(K.sum(class_prec_tensor), 'float32') / K.cast(K.maximum(K.sum(precision_mask), 1), 'float32')
        return class_prec
    return prec

In [0]:
# 특정 클래스에 대한 재현율
def single_class_recall(interesting_class_id):
    def recall(y_true, y_pred):
        class_id_true = K.argmax(y_true, axis=-1)
        class_id_pred = K.argmax(y_pred, axis=-1)
        recall_mask = K.cast(K.equal(class_id_true, interesting_class_id), 'int32')
        class_recall_tensor = K.cast(K.equal(class_id_true, class_id_pred), 'int32') * recall_mask
        class_recall = K.cast(K.sum(class_recall_tensor), 'float32') / K.cast(K.maximum(K.sum(recall_mask), 1), 'float32')
        return class_recall
    return recall

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 256, weights=[embedding_matrix], input_length=max_len, trainable=True)) # 오류가 난다면 차원을 변경하세요.
#model.add(Flatten()) # Dense의 입력으로 넣기위함.
model.add(Dropout(0.2))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(20, activation='softmax'))

In [0]:
model = Sequential()
model.add(e) 
model.add(Flatten()) # Dense의 입력으로 넣기위함.
model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='softmax'))

In [0]:
model.summary()

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy',
                       single_class_precision(0), single_class_recall(0),
                       single_class_precision(1), single_class_recall(1),
                       single_class_precision(2), single_class_recall(2),
                       single_class_precision(3), single_class_recall(3),
                       single_class_precision(4), single_class_recall(4),
                       single_class_precision(5), single_class_recall(5),
                       single_class_precision(6), single_class_recall(6),
                       single_class_precision(7), single_class_recall(7),
                       single_class_precision(8), single_class_recall(8),
                       single_class_precision(9), single_class_recall(9),
                       single_class_precision(10), single_class_recall(10),
                       single_class_precision(11), single_class_recall(11),
                       single_class_precision(12), single_class_recall(12),
                       single_class_precision(13), single_class_recall(13),
                       single_class_precision(14), single_class_recall(14),
                       single_class_precision(15), single_class_recall(15),
                       single_class_precision(16), single_class_recall(16),
                       single_class_precision(17), single_class_recall(17),
                       single_class_precision(18), single_class_recall(18),
                       single_class_precision(19), single_class_recall(19)])

In [0]:
model.fit(input_train, label_train, epochs=15, verbose=1, validation_data=(input_val, label_val)) # validation_data는 각 훈련마다 결과값을 도출할 측정 데이터를 의미한다. 따라서 가중치 업데이트는 되지 않는다.

In [0]:
# 6. 모델 평가하기
loss_and_metrics = model.evaluate(input_test, label_test, batch_size=256)
print('## evaluation loss and_metrics ##')
print(loss_and_metrics)

In [0]:
model.predict_classes(input_data)

In [0]:
train_data['label'] = model.predict_classes(input_data)

In [0]:
train_data[25010:25020]

In [0]:
!pip install xlsxwriter

In [0]:
dataframe = pd.DataFrame(train_data)
dataframe.to_excel("/content/drive/My Drive/text/GloVeEmbedding_result(1024).xlsx", engine='xlsxwriter', index=False)

### 결과 보고

loss_and_metrics = model.evaluate(input_test, label_test, batch_size=256)
- 모델 평가에서의 배치사이즈를 256, 512로 설정하여 정답레이블-예측레이블 오차를 측장하여 비교해봤으나, 둘 다 39371개 중 810를 잘못 예측하였습니다. 
- 배치 사이즈가 1일 경우, 128일 경우 정밀도와 재현율이 현저하게 떨어졌습니다.
- 256, 512의 경우 완벽하게 정확하게 예측이 일치하는 것으로 보아, 과대적합이 발생하여 훈련데이터에 적응해버린 것이 아닌가 추측됩니다. 
-- 잘못 생각한 것이었습니다. 평가층에서 배치 사이즈를 변경한다고하여, 이미 훈련된 모델로 predict할 땐 전혀 영향을 끼치지 않습니다. 단치 배치 사이즈에 따른 모델을 평가하는 인자인 것 같습니다. 
-- 따라서 평가에서의 배치 사이즈가 아닌 glove 임베딩 층의 차원을 변경하여 어떤 결과가 나오는지 확인해보곘습니다.
- 일반적인 자체 데이터셋으로 임베딩층을 설정했을 때와, Pre-Trained GloVe 임베딩 층을 사용했을 때 결과가 별반 다르지 않았습니다.

### 추가 검사 방안
위의 코드는 glove.100d 파일을 이용하여 100차원의 임베딩 층을 생성하였습니다.
glove.300d 파일을 이용하여 300차원의 임베딩 층을 사용하여 차이가 있는지 확인해보곘습니다.

In [0]:
import numpy as np
embedding_dict = dict()
f = open('/content/drive/My Drive/datt/glove.6B.300d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 300개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()
print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

In [0]:
print(embedding_dict['respectable'])
print(len(embedding_dict['respectable']))

In [0]:
embedding_matrix = np.zeros((vocab_size, 300))
# 단어 집합 크기의 행과 300개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

In [0]:
print(t.word_index.items())

In [0]:
for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어를 1개씩 꺼내온다.
    temp = embedding_dict.get(word) # 단어(key) 해당되는 임베딩 벡터의 300개의 값(value)를 임시 변수에 저장
    if temp is not None:
        embedding_matrix[i] = temp # 임수 변수의 값을 단어와 맵핑되는 인덱스의 행에 삽입

In [0]:
# 특정 클래스에 대한 정밀도
def single_class_precision(interesting_class_id):
    def prec(y_true, y_pred):
        class_id_true = K.argmax(y_true, axis=-1) # y_true: 실제 값, 티아노 및 텐스플로우의 텐서(tensor)
        class_id_pred = K.argmax(y_pred, axis=-1) # y_pred: 예측 값, 티아노 및 텐스플로우의 텐서(tensor)
        precision_mask = K.cast(K.equal(class_id_pred, interesting_class_id), 'int32')
        class_prec_tensor = K.cast(K.equal(class_id_true, class_id_pred), 'int32') * precision_mask
        class_prec = K.cast(K.sum(class_prec_tensor), 'float32') / K.cast(K.maximum(K.sum(precision_mask), 1), 'float32')
        return class_prec
    return prec

In [0]:
# 특정 클래스에 대한 재현율
def single_class_recall(interesting_class_id):
    def recall(y_true, y_pred):
        class_id_true = K.argmax(y_true, axis=-1)
        class_id_pred = K.argmax(y_pred, axis=-1)
        recall_mask = K.cast(K.equal(class_id_true, interesting_class_id), 'int32')
        class_recall_tensor = K.cast(K.equal(class_id_true, class_id_pred), 'int32') * recall_mask
        class_recall = K.cast(K.sum(class_recall_tensor), 'float32') / K.cast(K.maximum(K.sum(recall_mask), 1), 'float32')
        return class_recall
    return recall

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

In [0]:
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Flatten, Dropout
from keras.layers import Conv1D, GlobalMaxPooling1D

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=True)) # 배치사이즈가 기존(128)보다 2배 커지면 어떻게 될까? => 
#model.add(Flatten()) # Dense의 입력으로 넣기위함.
model.add(Dropout(0.2))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(20, activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy',
                       single_class_precision(0), single_class_recall(0),
                       single_class_precision(1), single_class_recall(1),
                       single_class_precision(2), single_class_recall(2),
                       single_class_precision(3), single_class_recall(3),
                       single_class_precision(4), single_class_recall(4),
                       single_class_precision(5), single_class_recall(5),
                       single_class_precision(6), single_class_recall(6),
                       single_class_precision(7), single_class_recall(7),
                       single_class_precision(8), single_class_recall(8),
                       single_class_precision(9), single_class_recall(9),
                       single_class_precision(10), single_class_recall(10),
                       single_class_precision(11), single_class_recall(11),
                       single_class_precision(12), single_class_recall(12),
                       single_class_precision(13), single_class_recall(13),
                       single_class_precision(14), single_class_recall(14),
                       single_class_precision(15), single_class_recall(15),
                       single_class_precision(16), single_class_recall(16),
                       single_class_precision(17), single_class_recall(17),
                       single_class_precision(18), single_class_recall(18),
                       single_class_precision(19), single_class_recall(19)])

In [0]:
model.fit(input_train, label_train, epochs=15, verbose=1, validation_data=(input_val, label_val)) # validation_data는 각 훈련마다 결과값을 도출할 측정 데이터를 의미한다. 따라서 가중치 업데이트는 되지 않는다.

In [0]:
# 6. 모델 평가하기
loss_and_metrics = model.evaluate(input_test, label_test, batch_size=512)
print('## evaluation loss and_metrics ##')
print(loss_and_metrics)

In [0]:
#model.predict_classes(input_data)
train_data['label'] = model.predict_classes(input_data)

In [0]:
dataframe = pd.DataFrame(train_data)
dataframe.to_excel("/content/drive/My Drive/text/GloVeEmbedding_result(glove300d).xlsx", engine='xlsxwriter', index=False)

에측값을 label로 삽입한 엑셀 파일을 다운받아, 가지고 잇던 정답 레이블과 비교해보니 39370개 데이터 중 541개 데이터(문장) 예측에 실패했습니다. 
무려 98.65586%의 예측 정확도를 가지고 있습니다!
- 이미 정상적으로 각 레이블에 적합하도록 사전작업을 거친 데이터로 확인한 결과라, OCR 결과 텍스트를 그대로 사용할 경우의 정확도라고 말할 수 없습니다.